# Python과 텔레그램 봇 API를 이용해서 챗봇 만들기 실습 - 완전 기초

* 복작복잡 + 멜팅팟 공동 세미나 : 봇봇봇 - 나쁜 봇 랭봇, 텔레그램에서 보낸 한 철 
    - http://onoffmix.com/event/89407
    - [랭보, '지옥에서 보낸 한 철'](http://sitong.or.kr/bbs/zboard.php?id=recomend&page=30&sn1=&divpage=1&sn=off&ss=on&sc=on&select_arrange=name&desc=desc&no=49)
* 김무성

* 원소스 
    - 싸이랭 / 싸이랭 : 파트 3 - Python을 이용한 자연어처리 기초 실습
        - https://www.facebook.com/groups/psylang/

# 0. 싸이랭/랭봇 docker image pulling & container 실행

docker 환경 자체는 구성되어 있어야 한다.
* https://www.docker.com/

docker-compose
* https://docs.docker.com/compose/install/

실습 소스를 가져온다.

> git clone https://github.com/PsyLang/langbot.git 

docker-compose 스크립트를 실행시킨다.

> cd (설치경로)langbot/docker

해당디렉토리에 docker-compose.yml 설정 파일이 있다.

> # langbot/docker/docker-compose.yml
langbot:
  image: psylang/langbot:0.1
  ports:
    - "8888:8888" 
    - "6006:6006"
  volumes:
    - ../:/notebooks/work/langbot
  tty: true

스크립트 실행

> docker-compose up

웹 브라우저로 접속한다

> http://localhost:8888

# 1. @BotFather를 통한 봇 등록 / 토큰 발급 / 설정 

이제부터는 langbot/ui/telegram 디렉토리 안의 test_langbot_telegram.ipynb 파일 기준으로 진행된다.

### 텔레그램 봇 등록 & 토큰 발행

#### 참고
* [1] 텔레그램 로봇, 공식 API로 만들기 (파이썬, 구글 앱 엔진) - http://bakyeono.net/post/2015-08-24-using-telegram-bot-api.html

발급받은 토큰을 langbot.keyfile의 첫줄에 복사해넣는다.

In [ ]:
!ls ./

# 2. python-telegram-bot 설치

#### 참고
* [2] python-telegram-bot - https://github.com/python-telegram-bot/python-telegram-bot

이 실습에서는 python3 버전으로 설치한다.

In [ ]:
!pip3 install python-telegram-bot --upgrade

# 3. 간단한 에코 봇 만들기 

#### 참고
* [3] Extensions – Your first Bot - https://github.com/python-telegram-bot/python-telegram-bot/wiki/Extensions-%E2%80%93-Your-first-Bot

In [ ]:
import os

In [ ]:
from telegram.ext import Updater
from telegram.ext import CommandHandler
from telegram.ext import MessageHandler, Filters

In [ ]:
with open(os.path.join('.','langbot.keyfile'), 'r') as f :
        token = f.readline().strip()

In [ ]:
updater = Updater(token=token)

In [ ]:
dispatcher = updater.dispatcher

### 예제 1) 시작 메시지 등록

In [ ]:
def start(bot, update):
    bot.sendMessage(chat_id=update.message.chat_id, text="안녕, 난 언어학습을 도와주는 랭봇이야!")

In [ ]:
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

### 예제 2) 유저 챗 아이디 체크하고, 확인하는 핸들러 등록

In [ ]:
G_STATES = {}

In [ ]:
def init_state(chat_id) :
    G_STATES[chat_id] = '' 

In [ ]:
def check_state(chat_id) :
    if not chat_id in G_STATES:
        init_state(chat_id)

In [ ]:
def get_state(chat_id) :
    return G_STATES[chat_id]

In [ ]:
def set_state(chat_id, val) :
    G_STATES[chat_id] = val

In [ ]:
def iam(bot, update):
    chat_id = update.message.chat_id
    check_state(chat_id)
    st = get_state(chat_id)
    msg = "my chat_id is %d" %(chat_id)
    msg = msg + "\n" + "my state is %s" %(st)
    bot.sendMessage(chat_id=update.message.chat_id, text=msg)
    set_state(chat_id, st+str(1)) # 이 부분은 유저의 상태를 핸들링할 수 있다는 것을 보기 위해 추가되었다.

In [ ]:
iam_handler = CommandHandler('iam', iam)

In [ ]:
dispatcher.add_handler(iam_handler)

### 예제 3) 에코 봇으로 만들기 (대답 따라하는 메시지 핸들러 등록)

In [ ]:
def echo(bot, update):
    bot.sendMessage(chat_id=update.message.chat_id, text=update.message.text)

In [ ]:
echo_handler = MessageHandler(Filters.text, echo)
dispatcher.add_handler(echo_handler)

### 봇 실행

In [ ]:
updater.start_polling()

# 4. Supervisor를 이용해서 봇 스크립트를 계속 실행시키기

#### 참고자료
* [4] Building AI Chat bot using Python 3 & TensorFlow - https://speakerdeck.com/inureyes/building-ai-chat-bot-using-python-3-and-tensorflow
* [5] Supervisor: A Process Control System - http://supervisord.org/

### supervisor 설치

supervisor는 python2만 지원(아직은)

In [ ]:
!pip install supervisor

### 스크립트에 포함된 패키지들 설치

bs4 패키지가 설치되어야 한다

In [ ]:
!pip3 install bs4

### Supervisor 설정파일에 스크립트 등록

Supervisor에 봇 스크립트를 등록해서 실행시키기 위해서, supervisord.conf라는 파일에 몇가지 설정을 해야 한다. 그리고 특정 경로에 그 파일을 옮겨준다.

In [ ]:
!ls

In [ ]:
!cp supervisord.conf /etc/supervisord.conf

### Running Supervisor

터미널을 열고 (주피터 터미널을 쓰면 좀 더 쉬움) supervisor 명령 실행

> supervisord

그리고 다시 터미널에서 supervisorctl 명령 실행

> supervisorctl

# 참고자료 
* [1] 텔레그램 로봇, 공식 API로 만들기 (파이썬, 구글 앱 엔진) - http://bakyeono.net/post/2015-08-24-using-telegram-bot-api.html
* [2] python-telegram-bot - https://github.com/python-telegram-bot/python-telegram-bot
* [3] Extensions – Your first Bot - https://github.com/python-telegram-bot/python-telegram-bot/wiki/Extensions-%E2%80%93-Your-first-Bot
* [4] 
Building AI Chat bot using Python 3 & TensorFlow - https://speakerdeck.com/inureyes/building-ai-chat-bot-using-python-3-and-tensorflow
* [5] Supervisor: A Process Control System - http://supervisord.org/